# Projet E4 : Profils de Youtubeurs

## Partie Crowdfunding 

SOLERE Pauline, 
TOUATI Baptiste

In [30]:
import pandas as pd
import numpy as np
import csv
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen 
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from urlextract import URLExtract
import re
from datetime import datetime
import os, json
import time

La fonction suivante prend en paramètre un URL de vidéo youtube et retourne les liens de crowdfunding présent dans la description de la vidéo. Il est aussi possible de retourner seulement quelques sites de crowdfunding choisis en changeant les valeurs dans "list_crowdfunding" et "list_crowdfunding_avec_maj".

In [31]:
def function_get_link_donation(url): #la fonction prend en paramètre l'url d'une vidéo youtube
    req = Request(url , headers={'User-Agent' : 'Mozilla'})
    webpage = urlopen(req).read()
    page_soup = bs(webpage, "html.parser") #utilisation de BeautifulSoup pour récupérer la page web

    user_agent_list = [
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5)  AppleWebKit/537.36   (KHTML, like Gecko) Chrome/83.0.4103.97  Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64)        AppleWebKit/537.36   (KHTML, like Gecko) Chrome/83.0.4103.97  Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64)        AppleWebKit/537.36   (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)  AppleWebKit/537.36   (KHTML, like Gecko) Chrome/39.0.2171.95  Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5)  AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1       Safari/605.1.15",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0)     Gecko/20100101 Firefox/77.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
    ]
    
    #dans la code source de la page web on cherche toutes les balises HTML "script"
    one_a_tag = page_soup.findAll('script')
    one_a_tagS = str(one_a_tag) #trasnforme toute ce qu'il y a sous la basile script en string
    one_a_tag1 = one_a_tagS.split("\"description\":") #on isole ce qui se trouve avant et après description
    one_a_tag2 = one_a_tag1[1].split("}") #on prend tout ce qui est après description (première case) jusqu'à '}'
    
    resHttp = [i for i in range(len(one_a_tag2[0])) if one_a_tag2[0].startswith("http", i)] #dans la partie entre description et '}' on prend tout ce qui commence par http

    resN = [i for i in range(len(one_a_tag2[0])) if one_a_tag2[0].startswith(" ", i) or one_a_tag2[0].startswith("\\n", i)] #dans la partie entre description et '}' on prend tout ce qui commence par un espace ou un saut de ligne

    urls = []
    for x in resHttp:
        b = 1 #pour ne pas reprendre plusieurs fois le même lien on met une condition en plus sur b
        for y in resN:
            if b == 1 and x < y: #si le http est avant l'espace ou le saut de ligne
                s = slice(x,y) #alors on prend tout ce qu'il y a entre le http et l'espace ou le saut de ligne
                urls.append(one_a_tag2[0][s]) #on stocke le résultat précédent dans urls
                b = 0
                
    urls_final = []
    
    #La première liste de sites de crowdfunding est pour vérifier si dans les liens que nous avons trouvés 
    #précédemment certain sont des sites de crowdfunding (il n'y a donc pas de majuscule dans des URL)
    list_crowdfunding = ['tipeee','patreon','clipeee','utip','leetchi','ulule','kickstarter','kisskissbankbank','wiseed','indiegogo','lendopolis','wesharebonds','mymajorcompany','gofundme']
    
    #La deuxième liste est pour vérifier les sites avec 'bit' (qui sont des liens réduits), on regarde donc dans
    #le titre de la page source si cela correspond à un site de crowdfunding il faut donc avec toutes les écruitres possibles (avec ou sans majuscules)
    list_crowdfunding_avec_maj = ['tipeee','Tipeee','patreon','Patreon','clipeee','Clipeee','utip','uTip','leetchi','Leetchi','ulule','Ulule','kickstarter','Kickstarter','kisskissbankbank','KissKissBankBank','wiseed','Wiseed','indiegogo','Indiegogo','lendopolis','Lendopolis','wesharebonds','WeShareBonds','mymajorcompany','MyMajorCompany','gofundme','GoFundMe']

    for i in range(len(urls)):
        for j in range(len(list_crowdfunding)):
            if list_crowdfunding[j] in urls[i]: #si un des sites de crowdfunding est présent dans l'URL on le stocke dans urls_final
                urls_final.append(urls[i]) 

        if 'bit' in urls[i]: #ouvre la page source des lien avec 'bit' dedans et on regarde si le titre de la page correspond à un site de crowdfunding
            urli = urls[i]
            reqi = Request(urli , headers={'User-Agent': random.choice(user_agent_list)}) #varier les user-agent permet d'éviter d'être bloqué par le site web
            webpage = urlopen(reqi).read()
            page_soup = bs(webpage, "html.parser")
            one_a_tag_for = page_soup.findAll('title')
            for k in range(len(list_crowdfunding_avec_maj)):
                if list_crowdfunding_avec_maj[k] in one_a_tag_for:
                    urls_final.append(urls[i]) 
                
    #on met ensuite le résultat dans un data frame et on l'exporte en csv
    df_donation = pd.DataFrame({'lien crowdfundng': urls_final}) 
    df_donation.to_csv('function_get_donation.csv')
            
    return urls_final

Exemple avec un url quelconque pour montrer la sythaxe de div class="p-value system-txt" (pour tipeee) et span class = "blue-text semi-bold mr-1" (pour utip). Comme on peut le voir sur les résultats suivant pour Tipeee dans la classe p-value system-txt il y a deux information : les dons en premier et le nombre de Tipers en second. Pour uTip on a simplement l'information des abonnées dans la classe blue-text semi-bold mr-1. On se sert donc de ces informations trouvées sur dans la page source pour créer l'algorithme de la fonction suivante.

In [42]:
#TIPEEE
path = '/usr/local/bin/chromedriver'
options = Options() 
options.headless = True 
driver = webdriver.Chrome(executable_path = path, options=options)
driver.get('https://fr.tipeee.com/max-bird')
time.sleep(3)
soup_level1=bs(driver.page_source, 'lxml')
mydivs_class_tipeee = soup_level1.findAll("div", {"class": "p-value system-txt"})
print("Tipeee")
print(mydivs_class_tipeee)

#uTip
path = '/usr/local/bin/chromedriver'
options = Options() 
options.headless = True 
driver = webdriver.Chrome(executable_path = path, options=options)
driver.get("https://utip.io/polodebile")
time.sleep(3)
soup_level1=bs(driver.page_source, 'lxml')
mydivs_class_utip = soup_level1.findAll("span", {"class": "blue-text semi-bold mr-1"})
print("uTip")
print(mydivs_class_utip)

/var/folders/tw/7wfxm5gj5yncxf8mnytw_vqw0000gn/T/ipykernel_1537/88174564.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = path, options=options)


Tipeee
[<div class="p-value system-txt">491 €</div>, <div class="p-value system-txt">194</div>]


/var/folders/tw/7wfxm5gj5yncxf8mnytw_vqw0000gn/T/ipykernel_1537/88174564.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = path, options=options)


uTip
[<span class="blue-text semi-bold mr-1" data-v-f2780b1a="">525</span>]


La fonction suivante prend en argument un url de site de crownfunding, le résultat est l'url donné, le don affiché sur le site, le nombre d'abonnés affichés sur le site et le temps qui correspond à la date/heure où l'information a été prise sur le site. Cette fois-ci on utilise un web driver car les informations sont actualisés en temps réelle donc une autre technologie (Java) est utilisé par les sites de crowdfunding.

In [54]:
def function_get_sum_donation(url): 
    path = '/usr/local/bin/chromedriver' #il faut télécharger le webdriver selon la version chrome que l'on possède et donner le chemin où se site le webdriver téléchargé
    options = Options() #les options mises permettent d'éviter d'être bloquer par le navigateur
    options.headless = True 
    options.add_argument('--no-sandbox') 
    driver = webdriver.Chrome(executable_path = path, options=options)
    driver.get(url)
    time.sleep(3)
    
    found_don = []
    found_abo = []
    
    soup_level1=bs(driver.page_source, 'lxml')
    
    if 'tipeee' in url: #cas si l'url est un lien TIPEEE
        mydivs_class = soup_level1.findAll("div", {"class": "p-value system-txt"}) #on stocke toutes les valeurs de la classe voulue
        for i in range(len(mydivs_class)):
            str_mydivs_class = str(mydivs_class[i])
            if "€" in str_mydivs_class: #si il y a un € dans la "case" on stocke sa valeur dans tab_don car c'est une valeur monétaire
                str_tab_don = str(mydivs_class[i])
                found_don=re.search('>(.+?)<', str_tab_don).group(1)
            else: #sinon on stocke la valeur dans les abonnés
                str_tab_tiper = str(mydivs_class[i])
                found_abo=re.search('>(.+?)<', str_tab_tiper).group(1)
            
    if 'utip' in url: #cas si l'url est un lien uTip
        mydivs_class = soup_level1.findAll("span", {"class": "blue-text semi-bold mr-1"}) #on stocke toutes les valeurs de la classe voulue
        for i in range(len(mydivs_class)):
            str_tab_tiper = str(mydivs_class[i])
            found_abo=re.search('>(.+?)<', str_tab_tiper).group(1)
            
    time_now = str(datetime.now())
    driver.quit()
    return url, found_don, found_abo, time_now

On teste ensuite les deux fonctions sur un exemple concret : une vidéo de max bird.

In [81]:
url = 'https://www.youtube.com/watch?v=dayHLiePIXo'
urls_final = function_get_link_donation(url)

url_final = []
found_don_final = []
found_abo_final = []
time_now_final = []

for i in range(len(urls_final)):
    url_f, found_don, found_abo, time_now = function_get_sum_donation(urls_final[i])
    url_final.append(url_f)
    found_don_final.append(str(found_don))
    found_abo_final.append(found_abo)
    time_now_final.append(time_now)
    
df_url_don_abo_time=pd.DataFrame({'url': url_final,'Don': found_don_final, 'Abo': found_abo_final, 'Temps':time_now_final})
print('..................................')
print('Result 1 : function_get_link_donation')
print(urls_final)
print(" ")
print('Result 2 : function_get_sum_donation')
print(df_url_don_abo_time)
print('..................................')

/var/folders/tw/7wfxm5gj5yncxf8mnytw_vqw0000gn/T/ipykernel_1537/2437690394.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = path, options=options)


..................................
Result 1 : function_get_link_donation
['https://fr.tipeee.com/max-bird', 'https://www.clipeee.com/creator/max-bird)']
 
Result 2 : function_get_sum_donation
                                         url    Don  Abo  \
0             https://fr.tipeee.com/max-bird  491 €  194   
1  https://www.clipeee.com/creator/max-bird)     []   []   

                        Temps  
0  2022-04-18 10:39:40.420080  
1  2022-04-18 10:39:46.339427  
..................................


### Fichier excel WIZDEO 

L'objectif maintenant est de faire tourner nos fonctions créées sur des données réelles. Nous allons dans un premier temps utiliser le fichier excel 2021_10_08_FR_fr_Channels_LISIS_STUDENTS.xlsx et nottament la colonne summary. Celle-ci décrit la chaine et nous avons cherché si des liens de crowdfunding étaient présents dedans. 

In [56]:
fichier_chaine = pd.read_excel('2021_10_08_FR_fr_Channels_LISIS_STUDENTS.xlsx') #on ouvre le fichier excel
df = pd.DataFrame(fichier_chaine) #on le transforme en data frame
df_chaine_summary = pd.DataFrame({'display_name': fichier_chaine.display_name, 'Sommaire': fichier_chaine.summary}) #on garde seulement ce qu'on a besoin le display name et le summary

On cherche donc dans la colonne summary les liens tipeee. l'algorithme est sur le même principe que celui fait pour la description youtube. L'un des principal changement est l'utilisation de URLExtract() qui permet de sortir tous les url présent dans une chaine de caractère. Nous pouvons aussi sortir tous les liens de crowdfunding en changeant la variable 'list_crownfunding' mais nous avons préféré dans un premier temps nous concentrer sur tipeee. 

In [76]:
lien_crownfunding = []
chaine2_crownfunding = []
lien = []
crownfunding = []
chaine = []

#list_crownfunding = ['tipeee','tipeeestream','patreon','clipeee','utip','leetchi','ulule','kickstarter','kisskissbankbank','wiseed','indiegogo','lendopolis','wesharebonds','mymajorcompany','gofundme','paypal']
list_crownfunding = ['tipeee']
                     
for i in range(len(df_chaine_summary)): #on extrait tous les URL présent dans les descriptions de chaine
    extractor = URLExtract()
    urls = extractor.find_urls(str(df_chaine_summary.Sommaire[i]))
    lien_crownfunding.append(urls)
    chaine2_crownfunding.append(df_chaine_summary.display_name[i]) #on stocke les liens de crowdfunding d'un coté et les noms de chaine
df_chaine_lien = pd.DataFrame({'display_name': chaine2_crownfunding,'Lien_crownfunding': lien_crownfunding})
df_chaine_lien.to_csv('Chaine_Lien.csv')

for i in range(len(df_chaine_lien)): #on vérifie pour chaque lien trouvé si c'est un lien de crowdfunding Tipeee 
    for j in range(len(list_crownfunding)): 
        if list_crownfunding[j] in str(df_chaine_lien.Lien_crownfunding[i]): #pour chaque ligne on regarde s'il y a un site de crowdfunding dedans
            for k in range(len(df_chaine_lien.Lien_crownfunding[i])): #il y a parfois plusieurs liens pour une chaine on doit donc prendre dans ces liens seulement ceux de crowdfunding  
                if list_crownfunding[j] in str(df_chaine_lien.Lien_crownfunding[i][k]):
                    found = re.findall(list_crownfunding[j], str(df_chaine_lien.Lien_crownfunding[i])) #permet de trouver le nom du site de crowdfunding
                    crownfunding.append(found)
                    lien.append(df_chaine_lien.Lien_crownfunding[i][k])
                    chaine.append(df_chaine_lien.display_name[i])
            
df_chaine_crown_lien = pd.DataFrame({'display_name': chaine,'Crownfunding': crownfunding, 'Lien': lien})
df_chaine_crown_lien.to_csv('Chaine_Lien_crownfunding_f.csv')

In [77]:
df_chaine_crown_lien.head()

,display_name,Crownfunding,Lien
0,ScienceEtonnante,[tipeee],https://www.tipeee.com/science-etonnante
1,Kriss Papillon,[tipeee],https://www.tipeee.com/kriss
2,MaxEstLa,[tipeee],https://www.tipeeestream.com/maxestla/donation
3,Caljbeut,[tipeee],https://www.tipeee.com/caljbeut-cartoons-trashs
4,Laupok,[tipeee],tipeee.com/laupok


Une fois qu'on a trouver tous les sites de crowdfunding et plus particulièrement de Tipeee l'objectif est donc de trouver les don et les adonnées sur le site, nous allons donc utiliser l'algorithme créé au début.

In [85]:
don_tipeee = []
don_tiper = []
lien_tipeee = []
time_final = []
chaine_tipeee = []
crown_tipeee = []

for i in range(len(df_chaine_crown_lien.Lien)): # certain lien ne sont pas au format correct (pas de http ou https) on doit donc les rajouter ce qui manque
    if 'https://' not in df_chaine_crown_lien.Lien[i]:
        if 'http://' not in df_chaine_crown_lien.Lien[i]:
            df_chaine_crown_lien.Lien[i] = 'https://www.' + df_chaine_crown_lien.Lien[i]

    url_f, found_don_f, found_tiper_f, time_f = function_get_sum_donation(df_chaine_crown_lien.Lien[i]) #on utilise la fonction function_get_sum_donation pour avoir le don et les abonnés
    don_tipeee.append(found_don_f) #on stocke ensuite les résultats dans des listes
    don_tiper.append(found_tiper_f)
    time_final.append(time_f)
    lien_tipeee.append(url_f)
    chaine_tipeee.append(df_chaine_crown_lien.display_name[i])
    crown_tipeee.append(df_chaine_crown_lien.Crownfunding[i])
    
df_chaine_crown_lien_don=pd.DataFrame({'display_name': chaine_tipeee,'Crownfunding': crown_tipeee, 'Lien': lien_tipeee, 'Don': don_tipeee, 'Nb Tipers': don_tiper, 'Temps':time_final})
print(df_chaine_crown_lien_don)

/var/folders/tw/7wfxm5gj5yncxf8mnytw_vqw0000gn/T/ipykernel_1537/2437690394.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = path, options=options)


           display_name      Crownfunding  \
0      ScienceEtonnante          [tipeee]   
1        Kriss Papillon          [tipeee]   
2              MaxEstLa          [tipeee]   
3              Caljbeut          [tipeee]   
4                Laupok          [tipeee]   
..                  ...               ...   
69     PhœnixProduction          [tipeee]   
70           Rachou Off          [tipeee]   
71             Falkanox          [tipeee]   
72  Picoteen / Yoshinoo  [tipeee, tipeee]   
73  Tony et ses pensées          [tipeee]   

                                               Lien     Don Nb Tipers  \
0          https://www.tipeee.com/science-etonnante      []       446   
1                      https://www.tipeee.com/kriss  €1,322       300   
2    https://www.tipeeestream.com/maxestla/donation      []        []   
3   https://www.tipeee.com/caljbeut-cartoons-trashs    €215        64   
4                     https://www.tipeee.com/laupok      []        []   
..                   

In [88]:
df_chaine_crown_lien_don.to_csv('Chaine_Lien_Dons_crownfunding_tipers_time_summary_f.csv')

In [ ]:
df_chaine_crown_lien_don.head()

Ensuite on rassemble le fichier excel d'origine avec le fichier qu'on vient de créer pour avoir toutes les colonnes présentes et pouvoir mieux annalyser les données. On utilise donc merge pour rassembler les excels selon la colonne "disply_name" présent dans les deux fichiers.

In [86]:
file_name = 'chaine_crown_lien_don_f.xlsx'
file_name2 = '2021_10_08_FR_fr_Channels_LISIS_STUDENTS.xlsx'
df_chaine_crown_lien_don.to_excel(file_name) 

In [87]:
f3 = df_chaine_crown_lien_don.merge(df, on = "display_name",  how = "left") 
f3.to_excel("Results_f.xlsx", index = False) 

### FICHIER JSON crowd

Dans cette partie nous allons utiliser les fichiers JSON donnés par WIZDEO et qui possèdent des liens de crowdfundig dedans. Ces liens sont issus d'une certains nombre de vidéo youtube.

In [90]:
# lister tous les fichiers json présent dans le dossier donné par WIZDEO
path_to_json = '/Users/pauline/Downloads/crowd'
json_files = [json for json in os.listdir(path_to_json)]
print(json_files)

['crowd_827244.json', 'crowd_472023.json', 'crowd_492762.json', 'crowd_1194593.json', 'crowd_1638391.json', 'crowd_711734.json', 'crowd_807867.json', 'crowd_2196974.json', 'crowd_1937099.json', 'crowd_2196077.json', 'crowd_2276626.json', 'crowd_2209794.json', 'crowd_151536.json', 'crowd_2238163.json', 'crowd_488800.json', 'crowd_1056995.json', 'crowd_1196516.json', 'crowd_1601044.json', 'crowd_148890.json', 'crowd_297982.json', 'crowd_165252.json', 'crowd_1025804.json', 'crowd_404164.json', 'crowd_362589.json', 'crowd_850457.json', 'crowd_1022154.json', 'crowd_2299695.json', 'crowd_2194158.json', 'crowd_620126.json', 'crowd_389611.json', 'crowd_2168913.json', 'crowd_873458.json', 'crowd_683655.json', 'crowd_785549.json', 'crowd_754943.json', 'crowd_2192976.json', 'crowd_1244559.json', 'crowd_1234525.json', 'crowd_398407.json', 'crowd_345087.json', 'crowd_1896620.json', 'crowd_2294265.json', 'crowd_1724029.json', 'crowd_2091645.json', 'crowd_1267452.json', 'crowd_665701.json', 'crowd_23

In [91]:
data_f = []
part1 = []
nom_crowd = []
lien_crowd = []
data_json_part1 = []
id_yt = []

# récupération des wa_id pour l'identification des youtubeurs
for i in range(len(json_files)):
    data_json_files = str(json_files[i])
    one_a_tag_ = data_json_files.split("_") #on utilise split pour diviser la chaine de caractère et récupérer seulement le chiffre entre 'crowd_' et '.json'
    data_json_part1.append(one_a_tag_[1])
    data_json_part1_str = str(data_json_part1[i])
    one_a_tag_bis = data_json_part1_str.split(".")
    id_yt.append(one_a_tag_bis[0])
    #pour chaque fichier JSON on met tout ce qu'il y a dedans dans un tableau pour récpérer les liens de crowdfunding de chaque fichier
    with open('/Users/pauline/Downloads/crowd/'+json_files[i]) as mon_fichier:
        data = json.load(mon_fichier)
        data_f.append(data)
#on réalise une autre boucle for pour diviser la partie lien dans le fichier json, car celle-ci est composé du nom du site de crowdfunding et du lien
for j in range(len(data_f)):
    data_f_j = str(data_f[j])
    one_a_tag1 = data_f_j.split("'") 
    part1.append(one_a_tag1)
    nom_crowd.append(part1[j][1])
    lien_crowd.append(part1[j][3])

In [92]:
id_yt #résultat de la récupération des wa_id

['827244',
 '472023',
 '492762',
 '1194593',
 '1638391',
 '711734',
 '807867',
 '2196974',
 '1937099',
 '2196077',
 '2276626',
 '2209794',
 '151536',
 '2238163',
 '488800',
 '1056995',
 '1196516',
 '1601044',
 '148890',
 '297982',
 '165252',
 '1025804',
 '404164',
 '362589',
 '850457',
 '1022154',
 '2299695',
 '2194158',
 '620126',
 '389611',
 '2168913',
 '873458',
 '683655',
 '785549',
 '754943',
 '2192976',
 '1244559',
 '1234525',
 '398407',
 '345087',
 '1896620',
 '2294265',
 '1724029',
 '2091645',
 '1267452',
 '665701',
 '2321870',
 '471926',
 '1156970',
 '2283373',
 '434086',
 '2350362',
 '760533',
 '516597',
 '2242601',
 '613955',
 '255408',
 '2120584',
 '673704',
 '80300',
 '481898',
 '1776276',
 '1869257',
 '847605',
 '449963',
 '2234896',
 '174438',
 '1363319',
 '2162866',
 '2246879',
 '318992',
 '588867',
 '2319209',
 '270992',
 '496218',
 '204153',
 '1697394',
 '336710',
 '2247157',
 '357192',
 '458',
 '337785',
 '771',
 '2195765',
 '601991',
 '2256341',
 '406876',
 '1308529

In [93]:
print(len(json_files)) #nombre de fichier json

1456


In [94]:
### tableau avec wa_id, les nom de crown et les liens
lien_tipeee = []
crown_tipeee = []
id_yt_f = []

for i in range(len(lien_crowd)):
    lien_tipeee.append(lien_crowd[i])
    crown_tipeee.append(nom_crowd[i])
    id_yt_f.append(id_yt[i])
    
df_crown_lien_id=pd.DataFrame({'wa_id': id_yt_f, 'Crownfunding': crown_tipeee, 'Lien': lien_tipeee})
print('end')

end


In [95]:
df_crown_lien_id.Crownfunding.value_counts()

utip                574
tipeee              415
paypal              202
ulule                93
patreon              89
kickstarter          50
kisskissbankbank     31
tudigo                1
proarti               1
Name: Crownfunding, dtype: int64

Comme nous pouvons le voir d'après le résultat précédent les sites les plus utilisés sont uTip et Tipeee, nous allons donc nous concentrer ceux eux pour la suite.

### Seulement TIPEEE

In [96]:
# on isole les lien tipeee
df_crown_lien_id_tipeee = df_crown_lien_id[list(df_crown_lien_id.Crownfunding == 'tipeee')]

In [97]:
df_crown_lien_id_tipeee

,wa_id,Crownfunding,Lien
4,1638391,tipeee,https://www.tipeee.com/ben-n-coâ\x80\x8bâ\x80\...
8,1937099,tipeee,https://www.numerama.com/politique/737996-cont...
14,488800,tipeee,https://fr.tipeee.com/naturi-ebene
18,148890,tipeee,https://www.tipeee.com/snir-gaming
21,1025804,tipeee,https://fr.tipeee.com/herve-body-concept
...,...,...,...
1434,363,tipeee,http://tipeee.com/chocmiel
1436,1098242,tipeee,https://en.tipeee.com/mathrix
1442,2211364,tipeee,https://fr.tipeee.com/don-scrimocraft
1449,386861,tipeee,https://fr.tipeee.com/dycoshtv


On utilise l'algorithme function_get_sum_donation pour avoir le don et le nombre d'abonnés

In [ ]:
import time

lien_crowd = list(df_crown_lien_id_tipeee.Lien)
nom_crowd = list(df_crown_lien_id_tipeee.Crownfunding)
don_tipeee = []
don_tiper = []
lien_tipeee = []
time_final = []
crown_tipeee = []
id_yt_f = []

for i in range(len(lien_crowd)):
    
    url_f, found_don_f, found_tiper_f, time_f = function_get_sum_donation(lien_crowd[i])
    don_tipeee.append(found_don_f) 
    don_tiper.append(found_tiper_f)
    time_final.append(time_f)
    lien_tipeee.append(url_f)
    crown_tipeee.append(nom_crowd[i])
    id_yt_f.append(id_yt[i])
    print(i)
    print(lien_crowd[i])
    
df_crown_lien_don_tipers_tipeee=pd.DataFrame({'wa_id': id_yt_f, 'Crownfunding': crown_tipeee, 'Lien': lien_tipeee, 'Don': don_tipeee, 'Nb Tipers': don_tiper, 'Temps':time_final})
df_crown_lien_don_tipers_tipeee.to_csv('Lien_Dons_crownfunding_tipers_time_JSON_tipeee_f.csv')
df_crown_lien_don_tipers_tipeee.to_excel('Lien_Dons_crownfunding_tipers_time_JSON_tipeee_f.xlsx') 
print('end')


/var/folders/tw/7wfxm5gj5yncxf8mnytw_vqw0000gn/T/ipykernel_1537/2437690394.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = path, options=options)


0
https://www.tipeee.com/ben-n-coâ\x80\x8bâ\x80\x8bâ\x80\x8bâ\x80\x8bâ\x80\x8bâ\x80\x8bâ\x80\x8b
1
https://www.numerama.com/politique/737996-contenus-extremes-complotisme-utip-est-il-plus-ethique-que-tipeee.html
2
https://fr.tipeee.com/naturi-ebene
3
https://www.tipeee.com/snir-gaming
4
https://fr.tipeee.com/herve-body-concept
5
https://www.tipeee.com/wonder-hook
6
https://fr.tipeee.com/cypaubr
7
https://fr.tipeee.com/prise-at-sensation
8
https://fr.tipeee.com/yoshushikaofficiel
9
https://www.tipeee.com/sarayah-le-testeur-inde
10
https://www.tipeee.com/dk-1
11
https://www.tipeee.com/l-apaise
12
https://fr.tipeee.com/jurassikytb
13
https://www.tipeee.com/simplexpaleo
14
https://fr.tipeee.com/destination-camping-car
15
https://fr.tipeee.com/jeremy-compiegneâ\x80\x8b
16
https://www.tipeee.com/game-movie-land
17
https://en.tipeee.com/ayeah
18
https://www.tipeee.com/le-corps-la-maison-l-esprit
19
https://www.tipeee.com/qte
20
https://fr.tipeee.com/aude
21
https://www.tipeee.com/thomas-verna

### Seulement UTIP

On réalise les même étapes que précédemment mais pour uTip

In [ ]:
df_crown_lien_id_utip = df_crown_lien_id[list(df_crown_lien_id.Crownfunding == 'utip')]

In [ ]:
df_crown_lien_id_utip

In [ ]:
import time

lien_crowd = list(df_crown_lien_id_utip.Lien)
nom_crowd = list(df_crown_lien_id_utip.Crownfunding)
don_utip = []
don_abo = []
lien_utip = []
time_final = []
crown_utip = []
id_yt_f = []

for i in range(len(lien_crowd)):
    url_f, found_don_f, found_tiper_f, time_f = function_get_sum_donation(lien_crowd[i])
    don_utip.append(found_don_f) 
    don_abo.append(found_tiper_f)
    time_final.append(time_f)
    lien_utip.append(url_f)
    crown_utip.append(nom_crowd[i])
    id_yt_f.append(id_yt[i])
    
df_crown_lien_don_tipers_utip=pd.DataFrame({'wa_id': id_yt_f, 'Crownfunding': crown_utip, 'Lien': lien_utip, 'Don': don_utip, 'Nb Abo': don_abo, 'Temps':time_final})
df_crown_lien_don_tipers_utip.to_csv('Lien_Dons_crownfunding_tipers_time_JSON_utip_f.csv')
df_crown_lien_don_tipers_utip.to_excel('Lien_Dons_crownfunding_tipers_time_JSON_utip_f.xlsx') 
print('end')

### TOUTE (utip/tipeee et les autres platformes)

In [ ]:
#df_crown_lien_id.Lien[1352]

In [ ]:
indexNames = df_crown_lien_id[df_crown_lien_id['Lien'] == 'argent"}' ].index #certains liens ne sont pas conformes ou créé des erreurs, nous les avons donc enlevés
df_crown_lien_id.drop(indexNames , inplace=True)
indexNames = df_crown_lien_id[df_crown_lien_id['Lien'] == 'https://www.patreon.com/posts/47954104' ].index
df_crown_lien_id.drop(indexNames , inplace=True)
print("done")

In [ ]:
import time

lien_crowd = list(df_crown_lien_id.Lien)
nom_crowd = list(df_crown_lien_id.Crownfunding)
don_tipeee_utip = []
don_tiper_utip = []
lien_tipeee_utip = []
time_final = []
crown_tipeee_utip = []
id_yt_f = []

for i in range(len(lien_crowd)):
    url_f, found_don_f, found_tiper_f, time_f = function_get_sum_donation(lien_crowd[i])
    don_tipeee_utip.append(found_don_f) 
    don_tiper_utip.append(found_tiper_f)
    time_final.append(time_f)
    lien_tipeee_utip.append(url_f)
    crown_tipeee_utip.append(nom_crowd[i])
    id_yt_f.append(id_yt[i])
    print(i)
    print(lien_crowd[i])
        
df_crown_lien_don_tipers_tipeee_utip=pd.DataFrame({'wa_id': id_yt_f, 'Crownfunding': crown_tipeee_utip, 'Lien': lien_tipeee_utip, 'Don': don_tipeee_utip, 'Nb Tipers': don_tiper_utip, 'Temps':time_final})
df_crown_lien_don_tipers_tipeee_utip.to_csv('Lien_Dons_crownfunding_tipers_time_JSON_utip_tipeee_f.csv')
df_crown_lien_don_tipers_tipeee_utip.to_excel('Lien_Dons_crownfunding_tipers_time_JSON_utip_tipeee_f.xlsx')
print('end')


In [ ]:
df_crown_lien_don_tipers_tipeee_utip=pd.DataFrame({'wa_id': id_yt_f, 'Crownfunding': crown_tipeee_utip, 'Lien': lien_tipeee_utip, 'Don': don_tipeee_utip, 'Nb Tipers': don_tiper_utip, 'Temps':time_final})
df_crown_lien_don_tipers_tipeee_utip.to_csv('Lien_Dons_crownfunding_tipers_time_JSON_utip_tipeee_f.csv')
df_crown_lien_don_tipers_tipeee_utip.to_excel('Lien_Dons_crownfunding_tipers_time_JSON_utip_tipeee_f.xlsx')
print(len(df_crown_lien_don_tipers_tipeee_utip))